# SVMRank についての調査

ランク学習の１実装事例である「SVMRank」について調査しました。

こちらを参考にいたしました。

http://www.cs.cornell.edu/People/tj/svm_light/svm_rank.html

## (1) 概要

pairwise 手法によるランク学習の実装例であり、C言語により実装されております。

非商用目的に限り、利用は無償とのことです。

### (1-1) 教師データ

クエリーごとに、素性（featureベクトルと等価）と、そのランクを教師データとして用意します。

ランクの値が大きいほど、そのクエリーIDにおけるランキングが高い素性である、という意味になるようです。

教師データのフォーマット：(feature=10件とします)

| ランク | クエリーID | feature1 |  feature2 | ・・・ |  feature10 | 
| :---: | :---: | :---: |  :---: | :---: |  :---: | 
| 3 | qid:1 | 1:0.5 |  2:0.0 | ・・・ |  10:0.5 | 
| 2 | qid:1 | 1:0.2 |  2:1.0 | ・・・ |  10:0.1 | 
| 1 | qid:1 | 1:0.0 |  2:1.0 | ・・・ |  10:0.0 | 
| 3 | qid:2 | 1:0.0 |  2:0.1 | ・・・ |  10:0.2 | 
| 2 | qid:2 | 1:1.0 |  2:0.3 | ・・・ |  10:0.2 | 
| 1 | qid:2 | 1:1.0 |  2:0.5 | ・・・ |  10:1.0 | 


### (1-2) 学習

テストデータを SVMRank の学習処理に渡すと、pairwise 手法により学習を行い、モデルが生成されます。

テストデータのフォーマット：(前項の教師データのフォーマットと同一)

| ランク | クエリーID | feature1 |  feature2 | ・・・ |  feature10 | 
| :---: | :---: | :---: |  :---: | :---: |  :---: | 
| 1 | qid:9 | 1:0.0 |  2:0.0 | ・・・ |  10:0.0 | 
| 1 | qid:9 | 1:0.1 |  2:1.0 | ・・・ |  10:0.1 | 
| 1 | qid:9 | 1:0.5 |  2:0.0 | ・・・ |  10:0.3 | 

### (1-3) 予測


テストデータを SVMRank の予測処理に渡すと、テストデータについてのランキング・スコアが得られます。

予測結果データのフォーマット：(テストデータと並びが同じになるようです)

| ランキング・スコア |
| :---: |
| -0.5 |
| 0.5 |
| 1.5 |

ただしこのランキング・スコアは、純粋にテストデータとランキング結果の対応だけに使用される想定であり、その値自体が何らかの指標となるものではないようです。

（すなわち、テストデータをランキング・スコアの降順に並べ替えて、ランキングを得る・・・といった利用を想定している様子）

したがって、テストデータが１件しかないばあい、この手法の予測結果は意味を持たないかと存じます。

## (2) 環境準備

### (2-1) ファイルを取得して解凍

媒体の圧縮ファイルはこちら ( http://download.joachims.org/svm_rank/current/svm_rank.tar.gz ) に公開されております。

```
MacBookPro-makmorit-jp:SVMRank makmorit$ pwd
/Users/makmorit/GitHub/donusagi-bot/learning/prototype/rank_learning/SVMRank
MacBookPro-makmorit-jp:SVMRank makmorit$ ls -al
total 864
drwxr-xr-x  3 makmorit  staff     102 May  2 13:16 .
drwxr-xr-x  6 makmorit  staff     204 May  2 13:16 ..
-rw-r--r--@ 1 makmorit  staff  440320 Apr 30 14:36 svm_rank.tar
MacBookPro-makmorit-jp:SVMRank makmorit$ tar -xvf svm_rank.tar
x LICENSE.txt
x Makefile
x svm_struct_api.h
x svm_struct_api.c
x svm_struct_api_types.h
x svm_struct_learn_custom.c
x svm_struct/Makefile
x svm_struct/svm_struct_main.c
x svm_struct/svm_struct_learn.h
x svm_struct/svm_struct_learn.c
x svm_struct/svm_struct_common.h
x svm_struct/svm_struct_common.c
x svm_struct/svm_struct_classify.c
x svm_light/LICENSE.txt
x svm_light/Makefile
x svm_light/svm_learn.c
x svm_light/kernel.h
x svm_light/svm_learn.h
x svm_light/svm_learn_main.c
x svm_light/svm_classify.c
x svm_light/svm_loqo.c
x svm_light/svm_common.c
x svm_light/svm_common.h
x svm_light/svm_hideo.c
MacBookPro-makmorit-jp:SVMRank makmorit$ ls -al
total 968
drwxr-xr-x  11 makmorit  staff     374 May  2 13:17 .
drwxr-xr-x   6 makmorit  staff     204 May  2 13:16 ..
-rwx------@  1 makmorit  staff    2327 Jul  4  2007 LICENSE.txt
-rwx------@  1 makmorit  staff    2252 Mar 15  2009 Makefile
drwxr-xr-x  13 makmorit  staff     442 May  2 13:17 svm_light
-rw-r--r--@  1 makmorit  staff  440320 Apr 30 14:36 svm_rank.tar
drwxr-xr-x   9 makmorit  staff     306 May  2 13:17 svm_struct
-rwx------@  1 makmorit  staff   24592 Mar 19  2009 svm_struct_api.c
-rwx------@  1 makmorit  staff    4039 Oct  9  2008 svm_struct_api.h
-rwx------@  1 makmorit  staff    5714 Mar 17  2009 svm_struct_api_types.h
-rwx------@  1 makmorit  staff    2126 Mar 15  2009 svm_struct_learn_custom.c
MacBookPro-makmorit-jp:SVMRank makmorit$ 
```

### (2-2) make実行

いくつか警告が出ますが無視できるようです。

```
MacBookPro-makmorit-jp:SVMRank makmorit$ make
cd svm_light; make svm_learn_hideo_noexe
gcc -c  -O3                      svm_learn_main.c -o svm_learn_main.o 
svm_learn_main.c:288:63: warning: illegal character encoding in string literal [-Winvalid-source-encoding]
  printf("    Kernel Methods - Support Vector Learning, B. Sch<F6>lkopf and C. Burges and\n");
                                                              ^~~~
1 warning generated.
gcc -c  -O3                      svm_learn.c -o svm_learn.o 
svm_learn.c:2860:24: warning: equality comparison with extraneous parentheses [-Wparentheses-equality]
      if((unlabeled[i] == 2)) {
          ~~~~~~~~~~~~~^~~~
svm_learn.c:2860:24: note: remove extraneous parentheses around the comparison to silence this warning
      if((unlabeled[i] == 2)) {
         ~             ^   ~
svm_learn.c:2860:24: note: use '=' to turn this equality comparison into an assignment
      if((unlabeled[i] == 2)) {
                       ^~
                       =
svm_learn.c:2866:29: warning: equality comparison with extraneous parentheses [-Wparentheses-equality]
      else if((unlabeled[i] == 3)) {
               ~~~~~~~~~~~~~^~~~
svm_learn.c:2866:29: note: remove extraneous parentheses around the comparison to silence this warning
      else if((unlabeled[i] == 3)) {
              ~             ^   ~
svm_learn.c:2866:29: note: use '=' to turn this equality comparison into an assignment
      else if((unlabeled[i] == 3)) {
                            ^~
                            =
2 warnings generated.
gcc -c  -O3                      svm_common.c -o svm_common.o 
svm_common.c:1009:12: warning: format string is not a string literal (potentially insecure) [-Wformat-security]
    printf(symbol);
           ^~~~~~
svm_common.c:1009:12: note: treat the string as an argument to avoid this
    printf(symbol);
           ^
           "%s", 
1 warning generated.
gcc -c  -O3                      svm_hideo.c -o svm_hideo.o 
cd svm_struct; make svm_struct_noexe
gcc -c  -O3 -fomit-frame-pointer -ffast-math -Wall  svm_struct_learn.c -o svm_struct_learn.o
gcc -c  -O3 -fomit-frame-pointer -ffast-math -Wall  svm_struct_classify.c -o svm_struct_classify.o
gcc -c  -O3 -fomit-frame-pointer -ffast-math -Wall  svm_struct_common.c -o svm_struct_common.o
gcc -c  -O3 -fomit-frame-pointer -ffast-math -Wall  svm_struct_main.c -o svm_struct_main.o
svm_struct_main.c:408:63: warning: illegal character encoding in string literal [-Winvalid-source-encoding]
  printf("    Kernel Methods - Support Vector Learning, B. Sch<F6>lkopf and C. Burges and\n");
                                                              ^~~~
1 warning generated.
gcc -c  -O3 -fomit-frame-pointer -ffast-math -Wall  svm_struct_api.c -o svm_struct_api.o
gcc -c  -O3 -fomit-frame-pointer -ffast-math -Wall  svm_struct_learn_custom.c -o svm_struct_learn_custom.o
gcc  -O3 -lm -Wall svm_struct/svm_struct_learn.o svm_struct_learn_custom.o svm_struct_api.o svm_light/svm_hideo.o svm_light/svm_learn.o svm_light/svm_common.o svm_struct/svm_struct_common.o svm_struct/svm_struct_main.o -o svm_rank_learn -L. -lm                    
gcc  -O3 -lm -Wall svm_struct_api.o svm_struct/svm_struct_classify.o svm_light/svm_common.o svm_struct/svm_struct_common.o -o svm_rank_classify -L. -lm                    
MacBookPro-makmorit-jp:SVMRank makmorit$ make clean
cd svm_light; make clean
rm -f *.o 
rm -f pr_loqo/*.o
rm -f svm_learn
rm -f svm_classify
rm -f libsvmlight.so
cd svm_struct; make clean
rm -f *.o *.tcov *.d core gmon.out *.stackdump
rm -f *.o *.tcov *.d core gmon.out *.stackdump 
MacBookPro-makmorit-jp:SVMRank makmorit$ ls -al
total 1504
drwxr-xr-x  14 makmorit  staff     476 May  2 13:19 .
drwxr-xr-x   6 makmorit  staff     204 May  2 13:18 ..
-rw-r--r--@  1 makmorit  staff    6148 May  2 13:18 .DS_Store
-rwx------@  1 makmorit  staff    2327 Jul  4  2007 LICENSE.txt
-rwx------@  1 makmorit  staff    2252 Mar 15  2009 Makefile
drwxr-xr-x  13 makmorit  staff     442 May  2 13:19 svm_light
-rw-r--r--@  1 makmorit  staff  440320 Apr 30 14:36 svm_rank.tar
-rwxr-xr-x   1 makmorit  staff   67804 May  2 13:19 svm_rank_classify
-rwxr-xr-x   1 makmorit  staff  196544 May  2 13:19 svm_rank_learn
drwxr-xr-x   9 makmorit  staff     306 May  2 13:19 svm_struct
-rwx------@  1 makmorit  staff   24592 Mar 19  2009 svm_struct_api.c
-rwx------@  1 makmorit  staff    4039 Oct  9  2008 svm_struct_api.h
-rwx------@  1 makmorit  staff    5714 Mar 17  2009 svm_struct_api_types.h
-rwx------@  1 makmorit  staff    2126 Mar 15  2009 svm_struct_learn_custom.c
MacBookPro-makmorit-jp:SVMRank makmorit$ 
```

## (3) example を使用した実行例

SVMRank のサイトで用意されている「example3.tar」( http://download.joachims.org/svm_light/examples/example3.tar.gz )を取得／解凍します。

```
MacBookPro-makmorit-jp:SVMRank makmorit$ tar -xvf example3.tar
x example3/
x example3/train.dat
x example3/test.dat
```

### (3-1) 学習処理

学習セットファイル train.dat を引数として学習処理を行うと、model という名のモデルファイルが生成されます。

```
MacBookPro-makmorit-jp:SVMRank makmorit$ ./svm_rank_learn -c 3 example3/train.dat example3/model 
Reading training examples...done
Training set properties: 5 features, 3 rankings, 12 examples
NOTE: Adjusted stopping criterion relative to maximum loss: eps=0.004667
Iter 1: ...*(NumConst=1, SV=1, CEps=4.6667, QPEps=0.0000)
Iter 2: ...*(NumConst=2, SV=2, CEps=0.9849, QPEps=0.0000)
Iter 3: ...*(NumConst=3, SV=2, CEps=0.8366, QPEps=0.0000)
Iter 4: ...*(NumConst=4, SV=2, CEps=0.4767, QPEps=0.1262)
Iter 5: ...*(NumConst=5, SV=3, CEps=0.0509, QPEps=0.0041)
Iter 6: ...*(NumConst=6, SV=4, CEps=0.0316, QPEps=0.0124)
Iter 7: ...*(NumConst=7, SV=4, CEps=0.0279, QPEps=0.0094)
Iter 8: ...*(NumConst=8, SV=4, CEps=0.0048, QPEps=0.0015)
Iter 9: ...(NumConst=8, SV=4, CEps=0.0015, QPEps=0.0015)
Final epsilon on KKT-Conditions: 0.00146
Upper bound on duality gap: 0.00499
Dual objective value: dval=2.23257
Primal objective value: pval=2.23756
Total number of constraints in final working set: 8 (of 8)
Number of iterations: 9
Number of calls to 'find_most_violated_constraint': 27
Number of SV: 4 
Norm of weight vector: |w|=1.88386
Value of slack variable (on working set): xi=0.15290
Value of slack variable (global): xi=0.15437
Norm of longest difference vector: ||Psi(x,y)-Psi(x,ybar)||=3.87313
Runtime in cpu-seconds: 0.03
Compacting linear model...done
Writing learned model...done
MacBookPro-makmorit-jp:SVMRank makmorit$ 
```

### (3-2) 予測処理

テストデータファイル test.dat を引数として予測処理処理を行うと、predictions という名の予測結果ファイルが生成されます。

```
MacBookPro-makmorit-jp:SVMRank makmorit$ ./svm_rank_classify example3/test.dat example3/model example3/predictions
Reading model...done.
Reading test examples...done.
Classifying test examples...done
Runtime (without IO) in cpu-seconds: 0.00
Average loss on test set: 0.0000
Zero/one-error on test set: 0.00% (1 correct, 0 incorrect, 1 total)
NOTE: The loss reported above is the fraction of swapped pairs averaged over
      all rankings. The zero/one-error is fraction of perfectly correct
      rankings!
Total Num Swappedpairs  :      0
Avg Swappedpairs Percent:   0.00
MacBookPro-makmorit-jp:SVMRank makmorit$
```

### (3-3) 予測結果を参照

predictions というファイルに、ランキング・スコアが出力されます。

（ちなみに、テストデータのレコードと同じ並びになっているようです）

これをユーザープログラムなどで降順に整列し、ランキング結果として利用する想定のようです。

````
MacBookPro-makmorit-jp:SVMRank makmorit$ cat example3/test.dat <---これはテストデータ
4 qid:4 1:1 2:0 3:0 4:0.2 5:1 #1
3 qid:4 1:1 2:1 3:0 4:0.3 5:0 #2
2 qid:4 1:0 2:0 3:0 4:0.2 5:1 #3
1 qid:4 1:0 2:0 3:1 4:0.2 5:0 #4
MacBookPro-makmorit-jp:SVMRank makmorit$ cat example3/predictions <---これは予測結果データ
2.45127674
1.41263953
0.92976471
-0.55576233

テストデータとランキング結果の対応
4 qid:4 1:1 2:0 3:0 4:0.2 5:1 #1 2.45127674
3 qid:4 1:1 2:1 3:0 4:0.3 5:0 #2 1.41263953
2 qid:4 1:0 2:0 3:0 4:0.2 5:1 #3 0.92976471
1 qid:4 1:0 2:0 3:1 4:0.2 5:0 #4 -0.55576233
```

#### 別のテストデータでも試行

テストデータ内のスコア値を、全て横並びの値 (=1) にしてみます。

```
MacBookPro-makmorit-jp:SVMRank makmorit$ ./svm_rank_classify example3/test2.dat example3/model example3/predictions2
Reading model...done.
Reading test examples...done.
Classifying test examples...done
Runtime (without IO) in cpu-seconds: 0.00
Average loss on test set: 0.0000
Zero/one-error on test set: 0.00% (1 correct, 0 incorrect, 1 total)
NOTE: The loss reported above is the fraction of swapped pairs averaged over
      all rankings. The zero/one-error is fraction of perfectly correct
      rankings!
Total Num Swappedpairs  :      0
Avg Swappedpairs Percent:   0.00
MacBookPro-makmorit-jp:SVMRank makmorit$ cat example3/test2.dat
1 qid:5 1:0 2:1 3:0 4:0.3 5:0 #1
1 qid:5 1:0 2:0 3:1 4:0.3 5:0 #2
1 qid:5 1:0 2:0 3:1 4:0.1 5:1 #3
1 qid:5 1:1 2:1 3:0 4:0.2 5:0 #4
MacBookPro-makmorit-jp:SVMRank makmorit$ cat example3/predictions2
-0.10887250
-0.57288748
0.42537783
1.42976468

テストデータとランキング結果の対応
1 qid:5 1:1 2:1 3:0 4:0.2 5:0 #4 1.42976468
1 qid:5 1:0 2:0 3:1 4:0.1 5:1 #3 0.42537783
1 qid:5 1:0 2:1 3:0 4:0.3 5:0 #1 -0.10887250
1 qid:5 1:0 2:0 3:1 4:0.3 5:0 #2 -0.57288748
```

### (3-4) モデルの評価

学習セットを予測処理の入力として予測を実行し、結果を評価するという流れは、他の予測手法と同様かと存じます。

```
MacBookPro-makmorit-jp:SVMRank makmorit$ ./svm_rank_classify example3/train.dat example3/model example3/predictions.eval
Reading model...done.
Reading test examples...done.
Classifying test examples...done
Runtime (without IO) in cpu-seconds: 0.00
Average loss on test set: 0.0000
Zero/one-error on test set: 0.00% (3 correct, 0 incorrect, 3 total)
NOTE: The loss reported above is the fraction of swapped pairs averaged over
      all rankings. The zero/one-error is fraction of perfectly correct
      rankings!
Total Num Swappedpairs  :      0
Avg Swappedpairs Percent:   0.00
MacBookPro-makmorit-jp:SVMRank makmorit$ cat example3/train.dat
# query 1
3 qid:1 1:1 2:1 3:0 4:0.2 5:0    <--- ランク#1 @ query 1
2 qid:1 1:0 2:0 3:1 4:0.1 5:1    <--- ランク#2 @ query 1
1 qid:1 1:0 2:1 3:0 4:0.4 5:0    <--- ランク#3 @ query 1
1 qid:1 1:0 2:0 3:1 4:0.3 5:0    <--- ランク#3 @ query 1
# query 2
1 qid:2 1:0 2:0 3:1 4:0.2 5:0    <--- ランク#2 @ query 2
2 qid:2 1:1 2:0 3:1 4:0.4 5:0    <--- ランク#1 @ query 2
1 qid:2 1:0 2:0 3:1 4:0.1 5:0    <--- ランク#2 @ query 2
1 qid:2 1:0 2:0 3:1 4:0.2 5:0    <--- ランク#2 @ query 2
# query 3
2 qid:3 1:0 2:0 3:1 4:0.1 5:1    <--- ランク#3 @ query 3
3 qid:3 1:1 2:1 3:0 4:0.3 5:0    <--- ランク#2 @ query 3
4 qid:3 1:1 2:0 3:0 4:0.4 5:1    <--- ランク#1 @ query 3
1 qid:3 1:0 2:1 3:1 4:0.5 5:0    <--- ランク#4 @ query 3
MacBookPro-makmorit-jp:SVMRank makmorit$ cat example3/predictions.eval
1.42976468    <--- ランク#1 @ query 1
0.42537783    <--- ランク#2 @ query 1
-0.12599765   <--- ランク#3 @ query 1
-0.57288748   <--- ランク#3 @ query 1
-0.55576233   <--- ランク#2 @ query 2
0.93149940    <--- ランク#1 @ query 2
-0.53863718   <--- ランク#2 @ query 2
-0.55576233   <--- ランク#2 @ query 2
0.42537783    <--- ランク#3 @ query 3
1.41263953    <--- ランク#1 @ query 3
2.41702644    <--- ランク#2 @ query 3
-0.66463483   <--- ランク#4 @ query 3
MacBookPro-makmorit-jp:SVMRank makmorit$
```

上記結果を見る限りでは、予測結果のランキング・スコアと、学習セットのランクの値が対応している（＝正しいモデルである）ことが確認できます。